In [142]:
import requests
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from PIL import Image
from io import BytesIO
from datetime import date
import xml.etree.ElementTree as ET


#### Requesting all drivers names

In [8]:
# Request
data = requests.request("GET", 'http://ergast.com/api/f1/drivers?limit=10000').text

# Parsing to DataFrame
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )

# Joining first and last name
pilots = pilots.assign(Name = pilots[['GivenName', 'FamilyName']].apply(' '.join, axis = 1)).drop(['GivenName', 'FamilyName'], axis = 1)

# Getting simillar drivers
# simmilar = process.extractBests("Lewis", pilots['Name'].to_list(), limit = 10, score_cutoff=80)

# a = []
# for i in simmilar:
#     a.append(i[0])
# tuple(a)

#### Getting wikipedia picture

In [9]:
# text = requests.request("GET", 'http://en.wikipedia.org/wiki/Lewis_Hamilton').text

# sample = "//upload.wikimedia.org/wikipedia/commons/thumb"

# start = text.find(sample)
# end = text[start:].find('"')

# 'https:' + text[start:(start + end)]

In [10]:
# text = requests.request("GET", pilots[pilots['Name'] == 'Lewis Hamilton']['url'].values[0]).text

# sample = "//upload.wikimedia.org/wikipedia/commons/thumb"

# start = text.find(sample)
# end = text[start:].find('"')

# 'https:' + text[start:(start + end)]

In [11]:
# url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Lewis_Hamilton_2016_Malaysia_2.jpg/640px-Lewis_Hamilton_2016_Malaysia_2.jpg'

# response = requests.get(url)

# Image.open(BytesIO(response.content))

#### Getting driver details

In [12]:
# Requesting data
data = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso').text

# Parsing to DataFrame
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )

#### Getting qualifying details

In [63]:
# Requesting data
quali = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso/qualifying?limit=1000').text

In [64]:
# Parsing to DataFrame
qualis = pd.read_xml(
    quali,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Race'
    )

# Parsing to Tree
tree = ET.fromstring(quali)

In [143]:
#RaceTable [0]
    #Race (season, round, url) [0][0]
        #RaceName {racename}  [0][0][0]
        #Circuit (circuitId, url) [0][0][1]
            #CircuitName {circuitname} [0][0][1][0]
            #Location (lat, long) [0][0][1][1]
                #Locality {Locality} [0][0][1][1][0]
                #Country {Country} [0][0][1][1][1]
        #Date {date} [0][0][2]
        #QualifyingList [0][0][3]
            #QualifyingResult (position) [0][0][3][0]
                #Driver [0][0][3][0][0]
                    #PermanentNumber {}[0][0][3][0][0][0]
                    #GivenName [0][0][3][0][0][1]
                    #FamilyName [0][0][3][0][0][2]
                    #DateOfBirth [0][0][3][0][0][3]
                    #Nationality [0][0][3][0][0][4]
                #Constructor [0][0][3][0][1]
                    #Name [0][0][3][0][1][0]
                    #Nationality

races = len(tree[0])

poles = 0
constructors = {}

nationality = tree[0][0].find('{http://ergast.com/mrd/1.5}QualifyingList').find('{http://ergast.com/mrd/1.5}QualifyingResult').find('{http://ergast.com/mrd/1.5}Driver').find('{http://ergast.com/mrd/1.5}Nationality').text
last_team = tree[0][races-1].find('{http://ergast.com/mrd/1.5}QualifyingList').find('{http://ergast.com/mrd/1.5}QualifyingResult').find('{http://ergast.com/mrd/1.5}Constructor').find('{http://ergast.com/mrd/1.5}Name').text

for i in range(races):
    if (
        tree[0][i]
            .find('{http://ergast.com/mrd/1.5}QualifyingList')
            .find('{http://ergast.com/mrd/1.5}QualifyingResult')
            .attrib['position'] == '1'):
        poles+=1

    constructor = tree[0][i].find('{http://ergast.com/mrd/1.5}QualifyingList').find('{http://ergast.com/mrd/1.5}QualifyingResult').find('{http://ergast.com/mrd/1.5}Constructor').find('{http://ergast.com/mrd/1.5}Name').text
    constructors[constructor] = None

print(nationality)
print(poles)
print(list(constructors.keys()))
print(last_team)
print(races)
print()

Spanish
23
['Minardi', 'Renault', 'McLaren', 'Ferrari', 'Alpine F1 Team']
Alpine F1 Team
326
True


#### Getting races details

In [154]:
# Requesting data
races_data = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso/results?limit=1000').text

In [155]:
# Parsing to Tree
tree = ET.fromstring(races_data)

In [156]:
races = len(tree[0])
wins = 0
podium = 0

for i in range(races):
    position = int(tree[0][i].find('{http://ergast.com/mrd/1.5}ResultsList').find('{http://ergast.com/mrd/1.5}Result').attrib['position'])
    wins = (wins + 1) if position == 1 else wins
    podium = (podium + 1) if position <= 3 else podium

print(races)
print(wins)
print(podium)

343
32
98


#### Getting championship details

In [179]:
# Requesting data
champioship_data = requests.request("GET", 'https://ergast.com/api/f1/drivers/hamilton/driverStandings/1').text

In [180]:
# Parsing to Tree
tree = ET.fromstring(champioship_data)

In [181]:
len(tree[0])

7

In [184]:
champioship_data = requests.request("GET", f'https://ergast.com/api/f1/drivers/senna/driverStandings').text

In [185]:
# Parsing to Tree
tree = ET.fromstring(champioship_data)

In [192]:
tree[0][0].attrib['season']

'1984'

In [201]:
results = []
wins = 0
for i in range(len(tree[0])):
    season = tree[0][i].attrib['season']
    position = tree[0][i].find('{http://ergast.com/mrd/1.5}DriverStanding').attrib['position'] 
    points = tree[0][i].find('{http://ergast.com/mrd/1.5}DriverStanding').attrib['points']
    emoji = '🥇' if position == '1' else '🥈' if position == '2' else '🥉' if position == '3' else ' '
    wins = wins+1 if position == '1' else wins
    results.append([season, position, points, emoji])

print(wins)
pd.DataFrame(results, columns=['season', 'position', 'points', 'medal'])

3


,season,position,points,medal
0,1984,9,13,
1,1985,4,38,
2,1986,4,55,
3,1987,3,57,🥉
4,1988,1,90,🥇
5,1989,2,60,🥈
6,1990,1,78,🥇
7,1991,1,96,🥇
8,1992,4,50,
9,1993,2,73,🥈


In [206]:
actual = requests.request("GET", f'https://ergast.com/api/f1/current/drivers/alonso/driverStandings').text
tree = ET.fromstring(actual)
position = tree[0][0][0].attrib['position']
points = tree[0][0][0].attrib['points']
wins = tree[0][0][0].attrib['wins']
